# Red Neuronal Profunda (DNN) para clasificación MNIST

Aplicaremos todos nuestros conocimientos para crear una DNN, frecuentemente llamada también una Artificial Neural Network (ANN).  El problema que vamos a trabajar se conoce como el "Hola Mundo" del aprendizaje profundo porque para la mayoría de estudiantes este es el primer algoritmo de aprendizaje profundo que ven.

El conjunto de datos se llama MNIST y se refiere al reconocimiento de dígitos escritos a mano.  Pueden encontrar más información en el sitio web de Yann LeCun (Director of AI Research, Facebook).  El es uno de los pioneros de todo este tema, así como de otras metodologías más complejas como las Redes Neurales Convolucionales (CNN) que se utilizan hoy día.

El conjunto de datos tiene 70,000 imágenes (28x28 pixels) de dígitos escritos a mano (1 dígito por imagen).

La meta es escribir un algoritmo que detecta qué dígito ha sido escrito.  Como solo hay 10 dígitos (0 al 9), este es un problema de clasificación con 10 clases.

Nuestra meta será construir una RN con 2 capas escondidas.

## Importar los paquetes relevantes

TensorFlow incluye un proveedor de datos de MNIST que utilizaremos acá.  Viene con el módulo **"tensorflow.keras.datasets"**.

In [25]:
import numpy as np
import tensorflow as tf
import pandas as pd
import time
from tensorflow.keras.optimizers import Adam

La siguiente instrucción, cuando se corre por primera vez, descarga el conjunto de datos en lo indicado por el parámetro path, relativo a  ~/.keras/datasets).  Como si se hubiera ejecutado Lo siguiente:

tf.keras.datasets.mnist.load_data(
    path='mnist.npz'
)

luego separa los datos en un conjunto para entrenamiento y otro para pruebas.

Si se ejecuta más de una vez, ya no descarga el archivo.

In [2]:
(X_entreno, y_entreno), (X_prueba, y_prueba) = tf.keras.datasets.mnist.load_data()
assert X_entreno.shape == (60000, 28, 28)
assert X_prueba.shape == (10000, 28, 28)
assert y_entreno.shape == (60000,)
assert y_prueba.shape == (10000,)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


## Datos

Esta sección es donde pre-procesaremos nuestros datos.

Por default, TF2 tiene conjuntos de datos de entrenamiento y de prueba, pero no tiene un conjunto de validación, por lo que debemos dividirlo por nuestra cuenta

Lo haremos del mismo tamaño que el conjunto de prueba

In [3]:
num_obs_validacion = y_prueba.shape[0]

Usaremos una variable dedicada para el número de muestras de prueba

In [4]:
num_obs_prueba = y_prueba.shape[0]

Generalmente preferimos "normalizar" nuestros datos en alguna forma para que el resultado sea numéricamente más estable.  En este caso simplemente preferimos tener entradas entre 0 y 1, por lo que definimos una función, que reciba la imagen MNIST.

Como los posibles valores de las entradas son entre 0 y 255 (256 posibles tonos de gris), al dividirlos por 255 obtenemos el resultado deseado.

In [5]:
X_entreno_normalizado = X_entreno / 255

Finalmente, normalizaremos y convertiremos los datos de pruebas en tandas.  Los normalizamos para que tengan la misma magnitud que los datos de entrenamiento y validación.

No hay necesidad de "barajearlo" ya que no estaremos entrenando con los datos de prueba.  Habra una sola tanda, igual al tamaño de los datos de prueba.

In [6]:
X_prueba_normalizado = X_prueba / 255

Una vez se han "normalizado" los datos, podemos proceder a extraer los datos de entrenamiento y de validación.

Nuestros datos de validación serán 10000 para ser igual al conjunto de prueba.

Finalmente, creamos una tanda con un tamaño de tanda igual al total de muestras de validación.

In [8]:
X_validacion = X_entreno_normalizado[-num_obs_validacion: , : , : ]
y_validacion = y_entreno[-num_obs_validacion:]

Similarmente, los datos de entrenamiento son todos los demás por lo que nos salteamos tantas observaciones como las hay en el conjunto de validación.

In [9]:
X_entreno = X_entreno_normalizado[ : X_entreno_normalizado.shape[0] - num_obs_validacion, : , : ]
y_entreno = y_entreno[ : y_entreno.shape[0] - num_obs_validacion]
num_obs_entreno = y_entreno.shape[0]

Convertir de Arreglos Numpy a Tensores

In [10]:
datos_entreno = tf.data.Dataset.from_tensor_slices((X_entreno, y_entreno))
datos_validacion = tf.data.Dataset.from_tensor_slices((X_validacion, y_validacion))
datos_prueba = tf.data.Dataset.from_tensor_slices((X_prueba, y_prueba))

Barajear y hacer tandas con el conjunto de datos de entrenamiento

In [11]:
TAMANIO_TANDA = 100
datos_entreno = datos_entreno.shuffle(buffer_size = num_obs_entreno).batch(TAMANIO_TANDA)

Hacer tandas con los conjunto de validación y prueba, no se necesita barajearlos

In [12]:
datos_validacion = datos_validacion.batch(TAMANIO_TANDA)
datos_prueba = datos_prueba.batch(TAMANIO_TANDA)

## Modelo

### Delineamos el modelo

Cuando pensamos sobre un algoritmo de aprenzaje profundo, casi siempre solo lo imaginamos.  Asi que esta vez, hagámoslo.  :)

In [13]:
tamanio_entrada = 784
tamanio_salida = 10

Usaremos el mismo ancho para ambas capas escondidas.  No es una necesidad!

In [14]:
tamanio_capa_escondida = 200

# Definimos cómo se verá el modelo

La primera capa (la de entrada):  cada observación es de 28x28 píxeles, por lo tanto es un tensor de rango 2.

Como aún no hemos aprendido sobre CNNs, no sabemos como alimentar este tipo de entrada a nuestra red, por lo tanto hay que "aplanar" las imágenes.  Hay un método conveniente **Flatten** que toma nuestro tensor de 28x28 y lo convierte en  un vector (None), o (784,)...porque 28x28 = 784.  Esto nos permite crear una red de alimentación hacia adelante.

    
**tf.keras.layers.Dense** básicamente implementa:  output = activation(dot(entrada, peso) + sesgo).  Requiere varios argumentos, pero los más importantes para nosotros son el ancho de la capa escondida y la función de activación.

La capa final no es diferente, solo nos aseguramos de activarla con **softmax**


In [15]:
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada

    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='tanh'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


### Seleccionar el optimizador y la función de pérdida

Definimos el optimizador que nos gustaría utilizar, la función de pérdida, y las métricas que nos interesa obtener en cada interacción

In [16]:
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Entrenamiento

Acá es donde entrenamos el modelo que hemos construído

Determinamos el número máximo de épocas.

Ajustamos el modelo , especificando:

* los datos de entrenamiento
* el número total de épocas
* y los datos de validación que creamos en el formato (entradas, metas)

In [17]:
NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS,
          validation_data = datos_validacion,
          verbose = 2)

Epoch 1/5
500/500 - 11s - 22ms/step - accuracy: 0.9186 - loss: 0.2754 - val_accuracy: 0.9593 - val_loss: 0.1344
Epoch 2/5
500/500 - 9s - 18ms/step - accuracy: 0.9673 - loss: 0.1076 - val_accuracy: 0.9697 - val_loss: 0.0963
Epoch 3/5
500/500 - 3s - 6ms/step - accuracy: 0.9784 - loss: 0.0692 - val_accuracy: 0.9729 - val_loss: 0.0898
Epoch 4/5
500/500 - 5s - 10ms/step - accuracy: 0.9852 - loss: 0.0477 - val_accuracy: 0.9762 - val_loss: 0.0812
Epoch 5/5
500/500 - 6s - 13ms/step - accuracy: 0.9895 - loss: 0.0340 - val_accuracy: 0.9773 - val_loss: 0.0884


## Probar el modelo

Como se discutió en clase, luego del entrenamiento (con los datos de entrenamiento), y la validación (con los datos de validación), probamos el potencial de predicción final de nuestro modelo con el conjunto de datos de prueba que el algoritmo NUNCA ha visto antes.

Es muy importante reconocer que estar "jugando" con los hiperparámetros sobre-ajusta el conjunto de datos de validación.

La prueba es la instancia absolutamente final. Nunca debe probarse el modelo antes de haber completamente ajustado el modelo.

Si se ajusta el modelo después de hacer la prueba, se empezará a sobre-ajustar el conjunto de datos de prueba, que echaría "por los suelos" el propósito original del mismo.

In [18]:
perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)

100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9692 - loss: 0.1350


In [19]:
# Si se desea, se puede aplicar un formateo "bonito"
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Pérdida de prueba: 0.12. Precisión de prueba: 97.14%


Utilizando el modelo inicial y los hiperparámetros dados en este notebook, la precisión de prueba final debe ser aproximadamente 97%.

Cada vez que se ejecuta el código, se obtiene una precisión diferente debido a la "barajeada" de las tandas, los pesos se inicializan en forma diferente, etc.

Finalmente, intencionalmente se ha llegado a una solución subóptima, para que puedan tener la oportunidad de mejorarla como ejercicio de laboratorio.

# Diferentes tamaños de tanda

In [24]:
# Lista para almacenar los resultados de cada tamaño de tanda
resultados = []

# Lista de tamaños de tanda para experimentar
tamanios_tanda = [10000, 1]

# Definición del modelo de red neuronal
modelo = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(tamanio_salida, activation='softmax')
])

# Configuración del modelo
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Iterar sobre cada tamaño de tanda
for tamanio in tamanios_tanda:
    # Preparar los conjuntos de datos
    datos_entreno = tf.data.Dataset.from_tensor_slices((X_entreno, y_entreno))
    datos_validacion = tf.data.Dataset.from_tensor_slices((X_validacion, y_validacion))
    datos_prueba = tf.data.Dataset.from_tensor_slices((X_prueba, y_prueba))

    # Configurar los datos en tandas
    datos_entreno = datos_entreno.shuffle(buffer_size=num_obs_entreno).batch(tamanio)
    datos_validacion = datos_validacion.batch(tamanio)
    datos_prueba = datos_prueba.batch(tamanio)

    # Entrenamiento del modelo
    print(f"Entrenando con tamaño de tanda '{tamanio}'...", end=" ")
    inicio = time.time()
    historia = modelo.fit(datos_entreno, epochs=NUMERO_EPOCAS, validation_data=datos_validacion, verbose=0)
    fin = time.time()
    duracion_entrenamiento = fin - inicio
    print("Finalizado!")

    # Evaluación del modelo en el conjunto de prueba
    perdida, precision = modelo.evaluate(datos_prueba)

    # Obtener precisión final de entrenamiento y validación
    precision_entrenamiento_final = historia.history['accuracy'][-1]
    precision_validacion_final = historia.history['val_accuracy'][-1]

    # Almacenar los resultados
    resultados.append({
        "Tamaño de tanda": tamanio,
        "Precisión de entrenamiento": "{:.2f}%".format(precision_entrenamiento_final * 100),
        "Precisión de prueba": "{:.2f}%".format(precision * 100),
        "Precisión de validación": "{:.2f}%".format(precision_validacion_final * 100),
        "Duración de entrenamiento (s)": "{:.2f}".format(duracion_entrenamiento)
    })

# Convertir los resultados en un DataFrame para su visualización
df_resultados = pd.DataFrame(resultados)
print(df_resultados)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Entrenando modelo con tamaño de tanda '10000'... Listo!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step - accuracy: 0.7898 - loss: 38.0445
Entrenando modelo con tamaño de tanda '1'... Listo!
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/step - accuracy: 0.9492 - loss: 55.6879


,Tamaño de tanda,Precisión de entrenamiento,Precisión de prueba,Precisión de validación,Tiempo de entrenamiento (s)
0,10000,76.62%,78.98%,81.09%,7.05
1,1,96.73%,95.79%,95.98%,502.28


# Diferentes tasas de apredizaje

In [28]:
# Lista para almacenar los resultados de cada tasa de aprendizaje
resultados = []

# Tasas de aprendizaje para evaluar
tasas_aprendizaje = [0.0001, 0.02]

# Definición del modelo de red neuronal
modelo = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(tamanio_salida, activation='softmax')
])

# Preparar los conjuntos de datos
datos_entreno = tf.data.Dataset.from_tensor_slices((X_entreno, y_entreno))
datos_validacion = tf.data.Dataset.from_tensor_slices((X_validacion, y_validacion))
datos_prueba = tf.data.Dataset.from_tensor_slices((X_prueba, y_prueba))

# Configurar los datos en tandas con tamaño fijo
TAMANIO_TANDA = 100
datos_entreno = datos_entreno.shuffle(buffer_size=num_obs_entreno).batch(TAMANIO_TANDA)
datos_validacion = datos_validacion.batch(TAMANIO_TANDA)
datos_prueba = datos_prueba.batch(TAMANIO_TANDA)

# Evaluar el modelo con diferentes tasas de aprendizaje
for tasa in tasas_aprendizaje:
    # Inicializar el optimizador con la tasa de aprendizaje actual
    optimizador = Adam(learning_rate=tasa)

    # Compilar el modelo con el optimizador configurado
    modelo.compile(optimizer=optimizador, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Entrenamiento del modelo
    print(f"Entrenando el modelo con tasa de aprendizaje '{tasa}'...", end=" ")
    inicio = time.time()
    historial = modelo.fit(datos_entreno, epochs=NUMERO_EPOCAS, validation_data=datos_validacion, verbose=0)
    fin = time.time()
    duracion_entrenamiento = fin - inicio
    print("Finalizado!")

    # Evaluar el modelo en el conjunto de prueba
    perdida, precision = modelo.evaluate(datos_prueba)

    # Obtener la precisión final en entrenamiento y validación
    precision_final_entrenamiento = historial.history['accuracy'][-1]
    precision_final_validacion = historial.history['val_accuracy'][-1]

    # Almacenar los resultados
    resultados.append({
        "Tasa de aprendizaje": tasa,
        "Precisión de entrenamiento": "{:.2f}%".format(precision_final_entrenamiento * 100),
        "Precisión de prueba": "{:.2f}%".format(precision * 100),
        "Precisión de validación": "{:.2f}%".format(precision_final_validacion * 100),
        "Duración de entrenamiento (s)": "{:.2f}".format(duracion_entrenamiento)
    })

# Crear un DataFrame para visualizar los resultados
df_resultados = pd.DataFrame(resultados)
print(df_resultados)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Entrenando el modelo con tasa de aprendizaje '0.0001'... Finalizado!
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9619 - loss: 21.9655
Entrenando el modelo con tasa de aprendizaje '0.02'... Finalizado!
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9514 - loss: 90.5542
   Tasa de aprendizaje Precisión de entrenamiento Precisión de prueba  \
0               0.0001                     98.20%              96.63%   
1               0.0200                     97.80%              95.69%   

  Precisión de validación Duración de entrenamiento (s)  
0                  96.77%                        106.78  
1                  95.99%                         94.02  


#precisión de validación de 98.5% o más

In [27]:
# Configuración de parámetros
TAMANIO_TANDA = 200
TAMANIO_CAPA_ESCONDIDA = 1024
TASA_APRENDIZAJE = 0.0006561
NUMERO_EPOCAS = 50

# Preparar los conjuntos de datos
datos_entreno = tf.data.Dataset.from_tensor_slices((X_entreno, y_entreno))
datos_validacion = tf.data.Dataset.from_tensor_slices((X_validacion, y_validacion))
datos_prueba = tf.data.Dataset.from_tensor_slices((X_prueba, y_prueba))

# Configurar los datos en tandas
datos_entreno = datos_entreno.shuffle(buffer_size=num_obs_entreno).batch(TAMANIO_TANDA)
datos_validacion = datos_validacion.batch(TAMANIO_TANDA)
datos_prueba = datos_prueba.batch(TAMANIO_TANDA)

# Definir el modelo con la nueva estructura
modelo = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),  # Capa de entrada
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(TAMANIO_CAPA_ESCONDIDA, activation='sigmoid'),  # Primera capa oculta
    tf.keras.layers.Dense(TAMANIO_CAPA_ESCONDIDA, activation='sigmoid'),  # Segunda capa oculta
    tf.keras.layers.Dense(tamanio_salida, activation='softmax')  # Capa de salida
])

# Crear el optimizador con la tasa de aprendizaje especificada
optimizador = Adam(learning_rate=TASA_APRENDIZAJE)

# Compilar el modelo
modelo.compile(optimizer=optimizador, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
print(f"Comenzando el entrenamiento con tamaño de tanda={TAMANIO_TANDA}, tamaño de capa oculta={TAMANIO_CAPA_ESCONDIDA}, tasa de aprendizaje={TASA_APRENDIZAJE}, épocas={NUMERO_EPOCAS}...")
inicio = time.time()
historial = modelo.fit(datos_entreno, epochs=NUMERO_EPOCAS, validation_data=datos_validacion, verbose=1)
fin = time.time()
duracion_entrenamiento = fin - inicio
print("Entrenamiento completado!")

# Evaluar el modelo en el conjunto de prueba
print("Evaluando el modelo...")
perdida, precision = modelo.evaluate(datos_prueba)

# Obtener precisión final en entrenamiento y validación
precision_final_entrenamiento = historial.history['accuracy'][-1]
precision_final_validacion = historial.history['val_accuracy'][-1]

# Almacenar los resultados
resultados = [{
    "Tasa de aprendizaje": TASA_APRENDIZAJE,
    "Precisión de entrenamiento": "{:.2f}%".format(precision_final_entrenamiento * 100),
    "Precisión de prueba": "{:.2f}%".format(precision * 100),
    "Precisión de validación": "{:.2f}%".format(precision_final_validacion * 100),
    "Duración del entrenamiento (s)": "{:.2f}".format(duracion_entrenamiento)
}]

# Mostrar resultados en un DataFrame
df_resultados = pd.DataFrame(resultados)
print(df_resultados)

Entrenando modelo con TAMANIO_TANDA=200, TAMANIO_CAPA_ESCONDIDA=1024, TASA_APRENDIZAJE=0.0006561, NUMERO_EPOCAS=50...
Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 19s 73ms/step - accuracy: 0.6657 - loss: 1.1409 - val_accuracy: 0.9173 - val_loss: 0.2875
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 19s 75ms/step - accuracy: 0.9138 - loss: 0.2942 - val_accuracy: 0.9303 - val_loss: 0.2380
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 18s 70ms/step - accuracy: 0.9272 - loss: 0.2495 - val_accuracy: 0.9409 - val_loss: 0.2061
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 23s 81ms/step - accuracy: 0.9379 - loss: 0.2087 - val_accuracy: 0.9501 - val_loss: 0.1776
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 19s 76ms/step - accuracy: 0.9468 - loss: 0.1760 - val_accuracy: 0.9530 - val_loss: 0.1694
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 17s 69ms/step - accuracy: 0.9554 - loss: 0.1507 - val_accuracy: 0.9573 - val_loss: 0.1464
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 21s 72ms/step - accuracy: 0.9606 - loss: 0.1310 - val_accuracy

,Tasa de aprendizaje,Precisión de entrenamiento,Precisión de prueba,Precisión de validación,Tiempo de entrenamiento (s)
0,0.0001,91.91%,92.36%,93.12%,13.06
1,0.0200,95.61%,94.09%,95.24%,12.69
2,0.0200,100.00%,97.86%,98.30%,960.17
